In [1]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_beforesunrise_add_rounds copy.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 6
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

context: 昏暗的灯光洒落在复古的木质桌面上，为酒吧营造出一种温馨而私密的空间。席琳低着头，手指轻轻地绕着酒杯边缘，眼神中流露出一丝忧郁。你静静地注视着席琳，心中涌起一股怜悯之情。

席琳的感情故事让你感同身受，你打算和席琳讲述你的感情故事，“席琳，其实我...”

actions:
1. "我也有类似的经历..."
2. "我能理解你的感受。"
3. "爱而不得的确很常见。"


I select option 3. TURN: 1 


context: 席琳抬起头来，深邃的眼眸里闪过一丝惊讶，随即又恢复了平静。她微微一笑，似乎对你的理解感到欣慰。"是啊，爱而不得或许是每个人都会经历的一课。"她轻轻地说，然后又陷入了沉思。你能感觉到，这个话题让你们之间的距离更近了一些。

actions:
1. 告诉席琳你的感情故事。
2. 询问席琳更多关于她的故事。
3. 继续谈论爱情的普遍性。

I select option 1. TURN: 2 


context: 你深吸一口气，开始讲述那段令人心酸的经历。"其实，我这次来欧洲，是因为结束了一段感情。"你的声音低沉而充满感情，席琳全神贯注地听着。"我攒了很久的钱，飞来马德里看望前女友，但发现她和别人在一起了。"你的话语里满是失落和伤感，席琳的眼神里充满了同情和理解。

summary: 在酒吧的温馨氛围中，你向席琳坦白了自己的感情故事，讲述了这次欧洲之行背后的真实原因。席琳听得非常认真，你们之间的距离因此变得更近。你的坦诚让这个晚上变得更加特别，让你们的关系更加深厚。